In [1]:
import numpy as np
from functools import cache
from itertools import zip_longest
import sys

%load_ext line_profiler

In [2]:
class Piece():
  rows: np.ndarray[np.uint8, np.uint8]
  cols: np.ndarray[np.uint8, np.uint8]
  rows_height: int
  rows_width: int
  cols_height: int
  cols_width: int

  def __init__(self, rows: tuple[tuple[bool]] | np.ndarray[np.uint8, np.uint8], cols: tuple[tuple[bool]] | np.ndarray[np.uint8, np.uint8]):
    rows = np.asarray(rows)
    cols = np.asarray(cols)

    # Trim piece to size
    while rows.size and ~np.any(rows[-1]) and cols.size and ~np.any(cols[-1]):
       rows = rows[:-1]
       cols = cols[:-1]

    while rows.size and ~np.any(rows[:,-1]) and cols.size and ~np.any(cols[:,-1]):
       rows = rows[:,:-1]
       cols = cols[:,:-1]
    
    # Edge case: when all rows or cols are trimmed from one side
    self.rows_height = 0 if not rows.size else rows.shape[0]
    self.rows_width = 0 if not rows.size else rows.shape[1]
    self.cols_height = 0 if not cols.size else cols.shape[0]
    self.cols_width = 0 if not cols.size else cols.shape[1]
    self.rows = np.packbits(rows, axis=0)
    self.cols = np.packbits(cols, axis=0)

  def __repr__(self):
    rows = np.unpackbits(self.rows, axis=0, count=self.rows_height)
    cols = np.unpackbits(self.cols, axis=0, count=self.cols_height)
    buffer = []
    for row, col in zip_longest(rows, cols):
        if row is not None:
          buffer.append(" " + " ".join('-' if elem else ' ' for elem in row))
        if col is not None:
          buffer.append(" ".join('|' if elem else ' ' for elem in col))
    return "\n".join(buffer)
  
  def __hash__(self):
     return hash((tuple(self.rows.flatten()), tuple(self.cols.flatten())))

  def __eq__(self, other):
      if isinstance(other, Piece):
          return np.array_equal(self.rows, other.rows) and np.array_equal(self.cols, other.cols)
      return NotImplemented
    
  def rotated(self):
    unpacked_rows = np.unpackbits(self.rows, axis=0, count=self.rows_height)
    unpacked_columns = np.unpackbits(self.cols, axis=0, count=self.cols_height)

    rows = np.rot90(unpacked_columns, 1)
    cols = np.rot90(unpacked_rows, 1)

    while rows.size and ~np.any(rows[0]) and cols.size and ~np.any(cols[0]):
      # top row is empty, so we can shift digit up
      rows = np.roll(rows, -1, 0)
      cols = np.roll(cols, -1, 0)

    return Piece(rows, cols)
  
  @cache
  def rotations(self):
    a = self
    b = a.rotated()
    c = b.rotated()
    d = c.rotated()
    return set((a, b, c, d))

pieces = [
  Piece(rows=((True, False), (True, False), (False, False)), cols=((True, True, False), (False, False, False))),
  Piece(rows=((False, False), (False, False), (False, False)), cols=((True, False, False), (True, False, False))),
  Piece(rows=((True, False), (True, False), (True, False)), cols=((False, True, False), (True, False, False))),
  Piece(rows=((True, False), (True, False), (True, False)), cols=((False, True, False), (False, True, False))),
  Piece(rows=((False, False), (True, False), (False, False)), cols=((True, True, False), (False, True, False))),
  Piece(rows=((True, False), (True, False), (True, False)), cols=((True, False, False), (False, True, False))),
  Piece(rows=((True, False), (True, False), (True, False)), cols=((True, False, False), (True, True, False))),
  Piece(rows=((True, False), (False, False), (False, False)), cols=((False, True, False), (False, True, False))),
  Piece(rows=((True, False), (True, False), (True, False)), cols=((True, True, False), (True, True, False))),
  Piece(rows=((True, False), (True, False), (True, False)), cols=((True, True, False), (False, True, False))),
]

print(sys.getsizeof(pieces[0].rows))

129


In [3]:
pieces[0]

 -
| |
 -

In [4]:
pieces[1]

|
|

In [5]:
pieces[9].rotations()

{   -
 | | |
  - -,
  -
 |  
  -
 | |
  -,
  -
 | |
  -
   |
  -,
  - -
 | | |
  -  }

In [6]:
pieces[0].rotations()

{ -
 | |
  -}

In [7]:
class Puzzle():
  ROWS_HEIGHT = 5
  ROWS_WIDTH = 5
  COLS_HEIGHT = 4
  COLS_WIDTH = 6
  rows: np.ndarray[np.uint8, np.uint8]
  cols: np.ndarray[np.uint8, np.uint8]

  def __init__(
      self,
      rows=np.packbits(np.zeros((ROWS_HEIGHT,ROWS_WIDTH), dtype=np.uint8), axis=0),
      cols=np.packbits(np.zeros((COLS_HEIGHT,COLS_WIDTH), dtype=np.uint8), axis=0),
    ):
    self.rows = rows
    self.cols = cols

  def __repr__(self):
    rows = np.unpackbits(self.rows, axis=0, count=self.ROWS_HEIGHT)
    cols = np.unpackbits(self.cols, axis=0, count=self.COLS_HEIGHT)
    buffer = []
    for row, col in zip_longest(rows, cols):
        if row is not None:
          buffer.append(" " + " ".join('-' if elem else '.' for elem in row))
        if col is not None:
          buffer.append(" ".join('|' if elem else '.' for elem in col))
    return "\n".join(buffer)

  def __hash__(self):
    return hash((tuple(self.rows.flatten()), tuple(self.cols.flatten())))

  def __eq__(self, other):
    if isinstance(other, Puzzle):
        return np.array_equal(self.rows, other.rows) and np.array_equal(self.cols, other.cols)
    return NotImplemented

  def with_piece(self, piece: Piece, v_offset: int, h_offset: int) -> "Puzzle":
    if (v_offset + piece.rows_height > self.ROWS_HEIGHT):
      return None
    if (h_offset + piece.rows_width > self.ROWS_WIDTH):
      return None
    if (v_offset + piece.cols_height > self.COLS_HEIGHT):
      return None
    if (h_offset + piece.cols_width > self.COLS_WIDTH):
      return None
    
    mask_rows = np.zeros_like(self.rows)
    if piece.rows.size:
      mask_rows[:,h_offset:h_offset+piece.rows.shape[1]] = piece.rows >> v_offset

    mask_cols = np.zeros_like(self.cols)
    if piece.cols.size:
      mask_cols[:,h_offset:h_offset+piece.cols.shape[1]] = piece.cols >> v_offset

    if np.any(np.bitwise_and(self.rows, mask_rows)):
      return None
    if np.any(np.bitwise_and(self.cols, mask_cols)):
      return None

    np.bitwise_or(self.rows, mask_rows, out=mask_rows)
    np.bitwise_or(self.cols, mask_cols, out=mask_cols)

    return Puzzle(mask_rows, mask_cols)
  
  def spaces_left(self):
    unpacked_rows = np.unpackbits(self.rows, axis=0, count=self.ROWS_HEIGHT)
    unpacked_cols = np.unpackbits(self.cols, axis=0, count=self.COLS_HEIGHT)
    return np.size(unpacked_cols) - np.count_nonzero(unpacked_cols) + np.size(unpacked_rows) - np.count_nonzero(unpacked_rows)

In [8]:
%timeit Puzzle().with_piece(pieces[7], 2, 3)

92.1 µs ± 16.8 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [9]:
%lprun -f Puzzle.with_piece Puzzle().with_piece(pieces[7], 2, 3)

Timer unit: 1e-07 s

Total time: 0.0004671 s
File: C:\Users\Vidminas\AppData\Local\Temp\ipykernel_9708\642494018.py
Function: with_piece at line 36

Line #      Hits         Time  Per Hit   % Time  Line Contents
    36                                             def with_piece(self, piece: Piece, v_offset: int, h_offset: int) -> "Puzzle":
    37         1         61.0     61.0      1.3      if (v_offset + piece.rows_height > self.ROWS_HEIGHT):
    38                                                 return None
    39         1         26.0     26.0      0.6      if (h_offset + piece.rows_width > self.ROWS_WIDTH):
    40                                                 return None
    41         1         25.0     25.0      0.5      if (v_offset + piece.cols_height > self.COLS_HEIGHT):
    42                                                 return None
    43         1         23.0     23.0      0.5      if (h_offset + piece.cols_width > self.COLS_WIDTH):
    44                            

In [10]:
Puzzle()

 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [11]:
Puzzle().spaces_left()

49

In [12]:
Puzzle().with_piece(pieces[0], 3, 4)

 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . -
. . . . | |
 . . . . -

In [13]:
Puzzle().with_piece(pieces[7].rotated(), 0, 0)

 - - . . .
| . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [14]:
Puzzle().with_piece(pieces[3], 0, 0).with_piece(pieces[4].rotated(), 0, 1)

 - - - . .
. | | . . .
 - - . . .
. | . . . .
 - . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [15]:
Puzzle().with_piece(pieces[9].rotated().rotated().rotated(), 1, 0)

 . . . . .
. . . . . .
 . - . . .
| | | . . .
 - - . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [16]:
# larger pieces ordered first, as this prunes more candidate solutions
pieces_order = (8, 9, 6, 5, 2, 0, 4, 3, 7, 1)
bad_puzzles = set()

# pick a puzzle piece
# choose rotation
# find top-left free corner
# put piece if it fits
# check if solved
def attempt(puzzle: Puzzle, used_pieces: dict[int, tuple]):
  if puzzle in bad_puzzles:
    return None
  if puzzle.spaces_left() == 2:
    return puzzle, used_pieces
  
  for i in pieces_order:
    if i in used_pieces:
      continue

    for rotation in pieces[i].rotations():
      for row in range(0, puzzle.ROWS_WIDTH):
        for col in range(0, puzzle.COLS_WIDTH):
          candidate = puzzle.with_piece(rotation, row, col)
    
          if candidate is not None:
            if candidate in bad_puzzles:
              continue

            candidate_used_pieces = used_pieces.copy()
            candidate_used_pieces[i] = (rotation, row, col)
            result = attempt(candidate, candidate_used_pieces)
            if result is not None:
              return result

  bad_puzzles.add(puzzle)
  return None

In [17]:
# A blank start does not have a unique solution, so this is just for profiling
# Interrupt manually after it runs for about a minute
%lprun -f attempt attempt(Puzzle(), {})

*** KeyboardInterrupt exception caught in code being profiled.

Timer unit: 1e-07 s

Total time: 58.6866 s
File: C:\Users\Vidminas\AppData\Local\Temp\ipykernel_9708\3463616618.py
Function: attempt at line 10

Line #      Hits         Time  Per Hit   % Time  Line Contents
    10                                           def attempt(puzzle: Puzzle, used_pieces: dict[int, tuple]):
    11      3401     556777.0    163.7      0.1    if puzzle in bad_puzzles:
    12                                               return None
    13      3401    1893581.0    556.8      0.3    if puzzle.spaces_left() == 2:
    14                                               return puzzle, used_pieces
    15                                             
    16     37367     318289.0      8.5      0.1    for i in pieces_order:
    17     33974     382145.0     11.2      0.1      if i in used_pieces:
    18     23761     127179.0      5.4      0.0        continue
    19                                           
    20     39414    3485531.0     88.4      0.6      for rotation 

In [18]:
# Puzzle 1 - Starter
bad_puzzles = set()
puzzle1_config = {
  1: (pieces[1], 2, 0),
  2: (pieces[2].rotated(), 1, 2),
  3: (pieces[3].rotated().rotated(), 2, 1),
  4: (pieces[4].rotated(), 0, 1),
  6: (pieces[6].rotated(), 3, 2),
  7: (pieces[7], 1, 4),
  9: (pieces[9].rotated(), 0, 3),
}
puzzle1 = Puzzle()
for starting_piece, v_offset, h_offset in puzzle1_config.values():
  puzzle1 = puzzle1.with_piece(starting_piece, v_offset, h_offset)
print(puzzle1)
print()

solution1 = attempt(puzzle1, puzzle1_config)
assert solution1 is not None
print(solution1[0])
assert solution1[1][8] == (pieces[8], 0, 0)
assert solution1[1][5] == (pieces[5], 2, 4)
assert solution1[1][0] == (pieces[0], 2, 2)

 . - - - -
. . | | | |
 . - - - -
. . | | | |
 . - . - .
| | . . . |
 . - . - .
| | | | | .
 . - - - .

 - - - - -
| | | | | |
 - - - - -
| | | | | |
 - - - - -
| | | | | |
 . - - - -
| | | | | |
 . - - - -


In [19]:
# Puzzle 2 - Starter
bad_puzzles = set()
puzzle2_config = {
  8: (pieces[8], 0, 0),
  4: (pieces[4].rotated(), 0, 1),
  3: (pieces[3].rotated(), 0, 3),
  9: (pieces[9].rotated().rotated().rotated(), 2, 0),
  5: (pieces[5], 1, 2),
  6: (pieces[6].rotated().rotated().rotated(), 1, 3),
  0: (pieces[0], 2, 4),
}
puzzle2 = Puzzle()
for starting_piece, v_offset, h_offset in puzzle2_config.values():
  puzzle2 = puzzle2.with_piece(starting_piece, v_offset, h_offset)
print(puzzle2)
print()

solution2 = attempt(puzzle2, puzzle2_config)
assert solution2 is not None
print(solution2[0])
assert solution2[1][7] == (pieces[7].rotated().rotated().rotated(), 3, 0)
assert solution2[1][1] == (pieces[1].rotated(), 4, 2)
assert solution2[1][2] == (pieces[2].rotated(), 3, 3)

 - - - - -
| | | | | |
 - - - - -
| | | | | |
 - - - - -
| | | | | |
 - - - . -
. . . . . .
 . . . . .

 - - - - -
| | | | | |
 - - - - -
| | | | | |
 - - - - -
| | | | | |
 - - - - -
. . | | | |
 - - - - -


In [20]:
# Puzzle 10 - Starter
bad_puzzles = set()
puzzle10_config = {
  6: (pieces[6], 2, 0),
  9: (pieces[9].rotated(), 2, 1),
  4: (pieces[4].rotated().rotated().rotated(), 3, 1),
  0: (pieces[0], 3, 3),
  5: (pieces[5], 2, 4),
}
puzzle10 = Puzzle()
for starting_piece, v_offset, h_offset in puzzle10_config.values():
  puzzle10 = puzzle10.with_piece(starting_piece, v_offset, h_offset)
print(puzzle10)
print()

solution10 = attempt(puzzle10, puzzle10_config)
assert solution10 is not None
print(solution10[0])
assert solution10[1][3] == (pieces[3].rotated(), 0, 0)
assert solution10[1][8] == (pieces[8].rotated(), 0, 3)
assert solution10[1][7] == (pieces[7].rotated(), 1, 0)
assert solution10[1][2] == (pieces[2].rotated(), 1, 2)
assert solution10[1][1] == (pieces[1], 1, 5)

 . . . . .
. . . . . .
 . . . . .
. . . . . .
 - - - . -
| | | | | .
 - - - - -
| | | | | |
 - - - - -



 - - . - -
| | | | | |
 - - - - -
| . | | | |
 - - - - -
| | | | | |
 - - - - -
| | | | | |
 - - - - -


In [21]:
# Puzzle 16 - Starter
bad_puzzles = set()
puzzle16_config = {
  2: (pieces[2], 2, 0),
  8: (pieces[8].rotated(), 2, 2),
  9: (pieces[9].rotated().rotated().rotated(), 0, 3),
}
puzzle16 = Puzzle()
for starting_piece, v_offset, h_offset in puzzle16_config.values():
  puzzle16 = puzzle16.with_piece(starting_piece, v_offset, h_offset)
print(puzzle16)
print()

solution16 = attempt(puzzle16, puzzle16_config)
assert solution16 is not None
print(solution16[0])
assert solution16[1][3] == (pieces[3].rotated(), 0, 0)
assert solution16[1][1] == (pieces[1].rotated(), 0, 2)
assert solution16[1][7] == (pieces[7].rotated(), 1, 0)
assert solution16[1][5] == (pieces[5].rotated(), 1, 1)
assert solution16[1][4] == (pieces[4], 1, 4)
assert solution16[1][0] == (pieces[0], 3, 1)
assert solution16[1][6] == (pieces[6].rotated(), 3, 3)

 . . . . -
. . . | | |
 . . . - -
. . . . . .
 - . - - .
. | | | | .
 - . - - .
| . . . . .
 - . . . .

 - - - - -
| | | | | |
 - - - - -
| | | | | |
 - - - - -
. | | | | |
 - - - - -
| | | | | |
 - - . - -


In [22]:
# Puzzle 112 - Master
bad_puzzles = set()
puzzle112_config = {
  6: (pieces[6], 0, 0),
  3: (pieces[3].rotated(), 0, 3),
  4: (pieces[4].rotated(), 3, 0),
  5: (pieces[5], 2, 4)
}
puzzle112 = Puzzle()
for starting_piece, v_offset, h_offset in puzzle112_config.values():
  puzzle112 = puzzle112.with_piece(starting_piece, v_offset, h_offset)
print(puzzle112)
print()

solution112 = attempt(puzzle112, puzzle112_config)
assert solution112 is not None
print(solution112[0])
assert solution112[1][0] == (pieces[0], 0, 1)
assert solution112[1][8] == (pieces[8].rotated(), 1, 2)
assert solution112[1][1] == (pieces[1], 2, 0)
assert solution112[1][2] == (pieces[2].rotated(), 2, 1)
assert solution112[1][7] == (pieces[7], 1, 4)
assert solution112[1][9] == (pieces[9].rotated().rotated().rotated(), 3, 2)

 - . . - -
| . . | | |
 - . . . .
| | . . . .
 - . . . -
. . . . | .
 - - . . -
. | . . . |
 - . . . -



 - - . - -
| | | | | |
 - - - - -
| | | | | |
 - - - - -
| | | | | |
 - - - - -
| | | | | |
 - . - - -


In [23]:
# Puzzle 120 - Master
bad_puzzles = set()
rot9 = pieces[9].rotated().rotated().rotated()
puzzle120 = Puzzle().with_piece(rot9, 1, 0)
print(puzzle120)
print()

solution120 = attempt(puzzle120, {9: (rot9, 1, 0)})
assert solution120 is not None
print(solution120[0])
# total bad solutions = 11115514

 . . . . .
. . . . . .
 . - . . .
| | | . . .
 - - . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .



KeyboardInterrupt: 

In [24]:
len(bad_puzzles)

6570

In [25]:
next(iter(bad_puzzles))

 - - . - .
| | | | | .
 - - . - .
| | | | | .
 - - . - -
| | | . | .
 . - . . -
| . | | . |
 . - - . -